In [1]:
import gc
import os 
import PIL
import torch
import pickle
import pandas as pd

import matplotlib.pyplot as plt

from IPython.display import clear_output
from matplotlib.patches import Rectangle
from collections import OrderedDict
from datetime import datetime

import utils.print as print_f

from utils.transforms import get_transform
from utils.engine import xami_train_one_epoch, xami_evaluate
from utils.engine import map_target_to_device
from utils.save import  get_train_data
from utils.plot import transparent_cmap, plot_result, plot_loss, get_legend_elements
from data.dataset import ReflacxDataset, collate_fn


## Suppress the assignement warning from pandas.
pd.options.mode.chained_assignment = None  # default='warn'


%matplotlib inline

In [11]:
import torchvision
torchvision.models.detection._utils.overwrite_eps

<function torchvision.models.detection._utils.overwrite_eps(model, eps)>

In [2]:
# Clean up memory.
gc.collect()
torch.cuda.empty_cache()
# torch.cuda.memory_summary(device=None, abbreviated=False)

In [3]:
XAMI_MIMIC_PATH = "D:\XAMI-MIMIC"

In [4]:
use_gpu = torch.cuda.is_available()
device = 'cuda' if use_gpu else 'cpu'
print(f"This notebook will running on device: [{device}]")

This notebook will running on device: [cuda]


In [5]:
disease_cmap = {
    "transparent": {
        "Enlarged cardiac silhouette": transparent_cmap(plt.cm.autumn),
        "Atelectasis": transparent_cmap(plt.cm.Reds),
        "Pleural abnormality": transparent_cmap(plt.cm.Oranges),
        "Consolidation": transparent_cmap(plt.cm.Greens),
        "Pulmonary edema": transparent_cmap(plt.cm.Blues),
    },
    "solid": {
        "Enlarged cardiac silhouette": "yellow",
        "Atelectasis": "red",
        "Pleural abnormality": "orange",
        "Consolidation": "lightgreen",
        "Pulmonary edema": "dodgerblue",
    },
}

legend_elements = get_legend_elements(disease_cmap["solid"])

In [6]:
labels_cols = [
    "Enlarged cardiac silhouette",
    "Atelectasis",
    "Pleural abnormality",
    "Consolidation",
    "Pulmonary edema",
    #  'Groundglass opacity', # 6th disease.
]

In [12]:
a = [2,3,4,5]

In [13]:
a[:-1]

[2, 3, 4]

In [7]:
detect_eval_params = {
    "XAMI_MIMIC_PATH": XAMI_MIMIC_PATH,
    "with_clinical":False,
    "dataset_mode": "unified",
    "bbox_to_mask":True,
    "labels_cols":labels_cols,
}

detect_eval_dataset = ReflacxDataset(
    **detect_eval_params,
    split_str="train",
    transforms=get_transform(train=True),
)

In [10]:
image, target = detect_eval_dataset[0]


cp1
tensor([[ 734., 1204., 2211., 2175.]], dtype=torch.float64)
cp2
tensor([[ 734., 1204., 2211., 2175.]], dtype=torch.float64)
cp3
tensor([[ 734., 1204., 2211., 2175.]], dtype=torch.float64)
cp4
tensor([[ 734., 1204., 2211., 2175.]], dtype=torch.float64)
cp5
tensor([[ 734., 1204., 2211., 2175.]], dtype=torch.float64)


In [ ]:
from utils.transforms import RandomHorizontalFlip

flip = RandomHorizontalFlip()

{'boxes': tensor([[ 333., 1204., 1810., 2175.]], dtype=torch.float64),
 'labels': tensor([1]),
 'image_id': tensor([0]),
 'area': tensor([1434167.], dtype=torch.float64),
 'iscrowd': tensor([0]),
 'dicom_id': '34cedb74-d0996b40-6d218312-a9174bea-d48dc033',
 'image_path': 'D:\\XAMI-MIMIC\\patient_18111516\\CXR-JPG\\s55032240\\34cedb74-d0996b40-6d218312-a9174bea-d48dc033.jpg',
 'masks': tensor([[[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8)}